# Laboratorio 9

### Rodolfo Rojas 20160315

In [1]:
#pip install nltk

In [2]:
#pip install textblob

In [56]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from textblob import TextBlob
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer 
import re
import string as str

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/joserodolforojasmazariegos/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joserodolforojasmazariegos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
data = pd.read_csv("Joker_tweets.csv",sep=',',encoding = "utf-8")

In [5]:
data.head()

,text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted,longitude,latitude
0,Είναι επίσημο: μας ξεπέρασαν στο #Joker factor...,False,0,NaN,2019-10-28T20:01:20Z,False,NaN,1188908628947935234,NaN,"<a href=""http://twitter.com/download/iphone"" r...",starman9,0,False,False,NaN,NaN
1,RT @JusticeLeaguerz: All the versions of the J...,False,0,NaN,2019-10-28T20:01:20Z,False,NaN,1188908626221719560,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",migTei,14,True,False,NaN,NaN
2,RT @PTimm82: Finished the Joker Poker Piece! I...,False,0,NaN,2019-10-28T20:01:19Z,False,NaN,1188908623600082944,NaN,"<a href=""http://twitter.com/download/iphone"" r...",TheParsa2,1635,True,False,NaN,NaN
3,the dance of freedom. the death bells. the ris...,False,0,NaN,2019-10-28T20:01:19Z,True,NaN,1188908622086103042,NaN,"<a href=""http://twitter.com/download/iphone"" r...",itzybitzyfitzy_,0,False,False,NaN,NaN
4,RT @chowell1210: LOOK AT WHAT MY LOCAL BEACH I...,False,0,NaN,2019-10-28T20:01:18Z,False,NaN,1188908620978806784,NaN,"<a href=""http://twitter.com/download/android"" ...",EB_Movies,25,True,False,NaN,NaN


### Funcion Normalizadora

In [170]:
def word_norm(text):
  text = text.str.lower()
  text = text.replace('.', '')
  text = text.replace(',', '')
  text = text.replace('!', '')
  text = text.replace(to_replace =r'!', value='',regex=True)
  text = text.replace('á', 'a')
  text = text.replace('é', 'e')
  text = text.replace('í', 'i')
  text = text.replace('ó', 'o')
  text = text.replace('ú', 'u')
  text = text.replace('?', '')
  text = text.replace('!', '')
  text = text.replace(to_replace =r'-', value='',regex=True)
  text = text.replace(')', '')
  text = text.replace('(', '')
  text = text.replace(':', '')
  text = text.replace('/', '')
  return(text)

### Adding New Columns

In [171]:
data['Handler']= '@'+data['screenName']
data['Source']=data.statusSource.str.extract(r'(>.*</a>)', expand=False)
data['Source']= data.Source.str.slice(start=1, stop= -4 )

In [172]:
data.head()

,text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted,longitude,latitude,Handler,Source
0,Είναι επίσημο: μας ξεπέρασαν στο #Joker factor...,False,0,NaN,2019-10-28T20:01:20Z,False,NaN,1188908628947935234,NaN,"<a href=""http://twitter.com/download/iphone"" r...",starman9,0,False,False,NaN,NaN,@starman9,Twitter for iPhone
1,RT @JusticeLeaguerz: All the versions of the J...,False,0,NaN,2019-10-28T20:01:20Z,False,NaN,1188908626221719560,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",migTei,14,True,False,NaN,NaN,@migTei,Twitter Web App
2,RT @PTimm82: Finished the Joker Poker Piece! I...,False,0,NaN,2019-10-28T20:01:19Z,False,NaN,1188908623600082944,NaN,"<a href=""http://twitter.com/download/iphone"" r...",TheParsa2,1635,True,False,NaN,NaN,@TheParsa2,Twitter for iPhone
3,the dance of freedom. the death bells. the ris...,False,0,NaN,2019-10-28T20:01:19Z,True,NaN,1188908622086103042,NaN,"<a href=""http://twitter.com/download/iphone"" r...",itzybitzyfitzy_,0,False,False,NaN,NaN,@itzybitzyfitzy_,Twitter for iPhone
4,RT @chowell1210: LOOK AT WHAT MY LOCAL BEACH I...,False,0,NaN,2019-10-28T20:01:18Z,False,NaN,1188908620978806784,NaN,"<a href=""http://twitter.com/download/android"" ...",EB_Movies,25,True,False,NaN,NaN,@EB_Movies,Twitter for Android


In [126]:
def corpus_creator(user,data):
    df= data[data['screenName']==user]
    df['text_length'] = df.text.str.len()
    df[['text']] = df.text.str.lower()
    out = df[['text','screenName','created','text_length']]
    return out

text             RT @thebambox: Limited art only from BAM! and ...
favorited                                                    False
favoriteCount                                                    0
replyToSN                                                      NaN
created                                       2019-10-28T19:39:56Z
truncated                                                    False
replyToSID                                                     NaN
id                                             1188903243193225222
replyToUID                                                     NaN
statusSource     <a href="https://twitter.com/TheAvengersZone" ...
screenName                                         TheAvengersZone
retweetCount                                                     3
isRetweet                                                     True
retweeted                                                    False
longitude                                                     

In [ ]:
polarity = []
subjectivity = []
dataX = data.head(500)

for tw in dataX['text']:
  tw = TextBlob(tw)
  ann = tw.sentiment
  polarity.append(ann[0])
  subjectivity.append(ann[1])
  